In [14]:
import logging
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import sys
import numpy as np
import pandas as pd
import torch
from ignite.engine import Events, create_supervised_evaluator, create_supervised_trainer
from ignite.handlers import EarlyStopping, ModelCheckpoint
from ignite.metrics import Accuracy

import monai
import tensorboard
from monai.data import ImageDataset, decollate_batch, DataLoader
from monai.handlers import StatsHandler, TensorBoardStatsHandler, stopping_fn_from_metric
from monai.transforms import EnsureChannelFirst, Compose, RandRotate90, Resize, ScaleIntensity

monai.config.print_config()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

MONAI version: 1.3.0
Numpy version: 1.24.3
Pytorch version: 2.1.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 865972f7a791bf7b42efbcd87c8402bd865b329e
MONAI __file__: c:\Users\<username>\anaconda3\Lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.12
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.1.0
scikit-image version: 0.20.0
scipy version: 1.11.1
Pillow version: 9.4.0
Tensorboard version: 2.14.1
gdown version: 4.7.1
TorchVision version: 0.16.0
tqdm version: 4.65.0
lmdb version: 1.4.1
psutil version: 5.9.0
pandas version: 2.0.3
einops version: 0.7.0
transformers version: 4.32.1
mlflow version: 2.7.1
pynrrd version: 1.0.0
clearml version: 1.13.1

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [49]:
#list of image paths
imgFolder = './LiverSegmentations'
images = []
for folders in os.listdir(imgFolder):
    for files in os.listdir(os.path.join(imgFolder, folders)):
        print(os.path.join(imgFolder, folders))
        imgPath = os.path.join(imgFolder, folders, files)
        images.append(imgPath)

folders = os.listdir(imgFolder)[1]
print(folders)
os.path.join(imgFolder, folders)

CRLM-CT-1002


'./LiverSegmentations\\CRLM-CT-1002'

In [16]:
#extract labels
labelPath = "./Colorectal Liver Metastases Clinical data April 2023.xlsx"
data = pd.read_excel(labelPath, sheet_name="CRLM (n=168)")
rec_5yr = data['Recurrence (5y)']
rec_2yr = data['Recurrence (2y)']
data_id = data['Patient-ID']
data_label = pd.concat([data_id, rec_2yr, rec_5yr], axis=1)

In [17]:
print(data_label.head())

     Patient-ID  Recurrence (2y)  Recurrence (5y)
0  CRLM-CT-1001                0                0
1  CRLM-CT-1002                1                1
2  CRLM-CT-1003                0                0
3  CRLM-CT-1004                0                0
4  CRLM-CT-1005                0                1


In [41]:
images = sorted(images)

[]


In [19]:
paths = []
for patients in data_label['Patient-ID']:
    for image in images:
        if image.find(patients) != -1:
            paths.append(image)

path_toadd = {'Path':paths}
paths_df = pd.DataFrame(path_toadd)
data_label = pd.concat([data_label, paths_df], axis=1)

In [20]:
labels_5yr = np.array(data_label['Recurrence (5y)'], dtype=np.int64)
labels_2yr = np.array(data_label['Recurrence (2y)'], dtype=np.int64)

In [21]:
from sklearn.model_selection import train_test_split

x_train2, x_val2, y_train2, y_val2 = train_test_split(data_label, labels_2yr, test_size=0.3)

In [40]:
print(len(x_train2))
print(len(x_val2))
print(data_label.head())

117
51
     Patient-ID  Recurrence (2y)  Recurrence (5y)  Path
0  CRLM-CT-1001                0                0   NaN
1  CRLM-CT-1002                1                1   NaN
2  CRLM-CT-1003                0                0   NaN
3  CRLM-CT-1004                0                0   NaN
4  CRLM-CT-1005                0                1   NaN


In [23]:
# define transforms
train_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize((512, 512, 96)), RandRotate90()])
val_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize((512, 512, 96))])

In [24]:
# define image dataset, data loader
check_ds = ImageDataset(image_files=np.array(x_train2['Path']), labels=np.array(y_train2), transform=train_transforms)
check_loader = DataLoader(check_ds, batch_size=2, num_workers=2, pin_memory=torch.cuda.is_available())
im, label = monai.utils.misc.first(check_loader)
print(type(im), im.shape, label)

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\HPatel33\anaconda3\Lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HPatel33\anaconda3\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HPatel33\anaconda3\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "c:\Users\HPatel33\anaconda3\Lib\site-packages\monai\data\image_dataset.py", line 108, in __getitem__
    img = self.loader(self.image_files[index])
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HPatel33\anaconda3\Lib\site-packages\monai\transform

In [25]:
# create DenseNet121, CrossEntropyLoss and Adam optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
net = monai.networks.nets.DenseNet264(spatial_dims=3, in_channels=1, out_channels=2).to(device)
loss = torch.nn.CrossEntropyLoss()
lr = 1e-5
opt = torch.optim.Adam(net.parameters(), lr)

cpu


In [26]:
print(torch.backends.cudnn.enabled)
print(torch.cuda.is_available())
print(torch.__version__)

True
False
2.1.0


In [27]:
# Ignite trainer expects batch=(img, label) and returns output=loss at every iteration,
# user can add output_transform to return other values, like: y_pred, y, etc.
trainer = create_supervised_trainer(net, opt, loss, device, False)

In [28]:

# adding checkpoint handler to save models (network params and optimizer stats) during training
checkpoint_handler = ModelCheckpoint("./runs_array/", "net", n_saved=10, require_empty=False)
trainer.add_event_handler(
    event_name=Events.EPOCH_COMPLETED, handler=checkpoint_handler, to_save={"net": net, "opt": opt}
)

In [29]:
# StatsHandler prints loss at every iteration and print metrics at every epoch,
# we don't set metrics for trainer here, so just print loss, user can also customize print functions
# and can use output_transform to convert engine.state.output if it's not loss value
train_stats_handler = StatsHandler(name="trainer", output_transform=lambda x: x)
train_stats_handler.attach(trainer)

In [30]:
# TensorBoardStatsHandler plots loss at every iteration and plots metrics at every epoch, same as StatsHandler
train_tensorboard_stats_handler = TensorBoardStatsHandler(output_transform=lambda x: x)
train_tensorboard_stats_handler.attach(trainer)

In [31]:
# set parameters for validation
validation_every_n_epochs = 1

In [32]:
metric_name = "Accuracy"
# add evaluation metric to the evaluator engine
val_metrics = {metric_name: Accuracy()}
# Ignite evaluator expects batch=(img, label) and returns output=(y_pred, y) at every iteration,
# user can add output_transform to return other values
evaluator = create_supervised_evaluator(net, val_metrics, device, True)


In [33]:
# add stats event handler to print validation stats via evaluator
val_stats_handler = StatsHandler(
    name="evaluator",
    output_transform=lambda x: None,  # no need to print loss value, so disable per iteration output
    global_epoch_transform=lambda x: trainer.state.epoch,
)  # fetch global epoch number from trainer
val_stats_handler.attach(evaluator)

In [34]:
# add early stopping handler to evaluator
early_stopper = EarlyStopping(patience=4, score_function=stopping_fn_from_metric(metric_name), trainer=trainer)
evaluator.add_event_handler(event_name=Events.EPOCH_COMPLETED, handler=early_stopper)

In [35]:
# create a validation data loader
val_ds = ImageDataset(image_files=np.array(x_val2['Path']), labels=np.array(y_val2), transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=2, num_workers=2, pin_memory=torch.cuda.is_available())

In [36]:
@trainer.on(Events.EPOCH_COMPLETED(every=validation_every_n_epochs))
def run_validation(engine):
    evaluator.run(val_loader)

In [37]:
# create a training data loader
train_ds = ImageDataset(image_files=np.array(x_train2['Path']), labels=np.array(y_train2), transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=2, pin_memory=torch.cuda.is_available())

In [38]:
train_epochs = 30
state = trainer.run(train_loader, train_epochs)
print(state)

INFO:ignite.engine.engine.Engine:Engine run starting with max_epochs=30.
ERROR:ignite.engine.engine.Engine:Current run is terminating due to exception: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\HPatel33\anaconda3\Lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HPatel33\anaconda3\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HPatel33\anaconda3\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "c:\Users\HPatel33\anaconda3\Lib\site-packages\monai\data\image_dataset.py", line 108, in __getitem__
    img = self.loader(

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\HPatel33\anaconda3\Lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HPatel33\anaconda3\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HPatel33\anaconda3\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "c:\Users\HPatel33\anaconda3\Lib\site-packages\monai\data\image_dataset.py", line 108, in __getitem__
    img = self.loader(self.image_files[index])
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HPatel33\anaconda3\Lib\site-packages\monai\transform